In [1]:
from manim import *
from PIL import Image
import matplotlib.pyplot as plt
import jupyter_capture_output

video_scene = " -v WARNING --disable_caching fe_Scene"
image_scene = f" -v WARNING --disable_caching -r {2*427},{2*240}  -s fe_Scene"

Jupyter Capture Output v0.0.11


In [104]:
# function to turn positions into complex numbers
def pos_to_complex(pos_array):
    N = len(pos_array)
    x_array = np.zeros((N,), dtype = np.complex256)
    for n in range(N):
        x_n_real = pos_array[n][0]
        x_n_imag = pos_array[n][1]
        x_array[n] = complex(x_n_real, x_n_imag)
    return x_array


# discrete fourier transform
def dft(x_array):
    N = len(x_array)
    k_array = np.zeros((N,), dtype = np.complex256)
    for k in range(N):
        for n in range(N):
            k_array[k] += x_array[n] * np.exp(-1j * 2*np.pi*k/N*n)
    return k_array

In [109]:
# reading image and grayscale conversion
image_dolphin = Image.open('../external_media/fourier_contour/dolphin_singleline.jpg').convert('L')
image_cheetah = Image.open('../external_media/fourier_contour/cheetah_singleline.jpg').convert('L')
image_elephant = Image.open('../external_media/fourier_contour/elephant_contour1_thin_manual.jpg').convert('L')
image_duo_elefant = Image.open('../external_media/fourier_contour/duo_elephant2.jpg').convert('L')
image_thiem = Image.open('../external_media/fourier_contour/thiem_edge3.png').convert('L')


# image processing
used_image = image_elephant
image_array = np.array(used_image)
n_height, n_width = image_array.shape
pixel_list = []

# define image size
y_min = 230
y_max = 230
x_min = 122
x_max = 122

for i_height in range(n_height):
    for i_width in range(n_width):
        if image_array[i_height, i_width] < 30:
            pixel_list.append(np.array([i_width, i_height]))
            if i_width > x_max:
                x_max = i_width
            if i_width < x_min:
                x_min = i_width
            if i_height > y_max:
                y_max = i_height
            if i_height < y_min:
                y_min = i_height


# fit the image to manim
x_range = x_max - x_min
y_range = y_max - y_min

x_manim_range = 14
y_manim_range = 8

x_manim_offset = 1
y_manim_offset = 1

n_points = len(pixel_list)

print(f"Number Points: {n_points}")
print(f"x-range: {x_min}, {x_max}")
print(f"y-range: {y_min}, {y_max}")


# transform pixel list to manim coordinates
coordinate_list = []
for x, y in pixel_list:
    # check image proportions to remain height / width ratio
    if x_range / x_manim_range > y_range / y_manim_range:
        x_point = (float(x) - x_min - x_range/2) / x_range * (x_manim_range-2*x_manim_offset)
        y_point = (float(y) - y_min - y_range/2) / x_range * (x_manim_range-2*y_manim_offset)
    else:
        x_point = (float(x) - x_min - x_range/2) / y_range * (y_manim_range-2*x_manim_offset)
        y_point = (float(y) - y_min - y_range/2) / y_range * (y_manim_range-2*y_manim_offset)
    coordinate_list.append((x_point, -y_point, 0))



# image_elephant

Number Points: 4430
x-range: 64, 991
y-range: 19, 978


In [110]:
# function to calculate the distance between two points
def get_distance(pos1, pos2):
    return np.sqrt( (pos1[0]-pos2[0])**2 + (pos1[1]-pos2[1])**2 + (pos1[2]-pos2[2])**2 )


# function to find the closest neighbor from a given coordinate and coordinate list
def get_closest_neighbor(input_coordinate, coordinate_list):
    # initial guess for closest neighbor
    min_dist = get_distance(input_coordinate, coordinate_list[0])
    min_dist_index = 0
    min_dist_coordinate = coordinate_list[0]
    # iterating though list to find actual closest neighbor
    for i, coordinate in enumerate(coordinate_list):
        dist = get_distance(input_coordinate, coordinate)
        # replace closest neighbor for even closer neighbor
        if dist < min_dist:
            min_dist = dist
            min_dist_index = i 
            min_dist_coordinate = coordinate
    # return closest neighbor
    return min_dist_coordinate, min_dist, min_dist_index


# function to insert a coordinate into a list into an appropriate position, return values: True (coordinate becomes new reference), False (standard)
def insert_coordinate(input_coordinate, coordinate_list):
    # initial guess for closest neighbor
    closest_neighbor = get_closest_neighbor(input_coordinate, coordinate_list)
    min_dist_index = closest_neighbor[2]
    # append new coordinate at the end (set input variable as reference: True)
    if min_dist_index == len(coordinate_list)-1:
        coordinate_list.append(input_coordinate)
        return True
    # insert new coordinata into list (reference remains untouched: False)
    else:
        pre_dist = get_distance(input_coordinate, coordinate_list[min_dist_index-1])
        post_dist = get_distance(input_coordinate, coordinate_list[min_dist_index+1])
        if pre_dist < post_dist:
            coordinate_list.insert(min_dist_index, input_coordinate)
        else:
            coordinate_list.insert(min_dist_index+1, input_coordinate)
        return False


# function to sort a list of coordinates (start is bottom left)
def sort_coordinate_list(coordinate_list, max_dist = x_manim_range):
    copy_coordinate_list = coordinate_list.copy()
    ordered_coordinate_list = []
    # find initial reference coordinate
    reference_coordinate = np.array([0, 0, 0])
    closest_neighbor = get_closest_neighbor(reference_coordinate, copy_coordinate_list)
    reference_coordinate = closest_neighbor[0]
    # ordering list items until list is empfty
    while (len(copy_coordinate_list)):
        # find closest neighbor and its properties
        closest_neighbor = get_closest_neighbor(reference_coordinate, copy_coordinate_list)
        min_dist_coordinate = closest_neighbor[0]
        min_dist = closest_neighbor[1]
        min_dist_index = closest_neighbor[2]
        # check for maximal distance violation
        if min_dist > max_dist:
            if insert_coordinate(min_dist_coordinate, ordered_coordinate_list):
                reference_coordinate = min_dist_coordinate
        else:
            ordered_coordinate_list.append(min_dist_coordinate)
            reference_coordinate = min_dist_coordinate
        # setting up the next iteration 
        copy_coordinate_list.pop(min_dist_index)
    return ordered_coordinate_list


# coordinate_list, 
ordered_coordinate_list_unrefined = sort_coordinate_list(coordinate_list) 
ordered_coordinate_list = sort_coordinate_list(coordinate_list, 3) 
print(f"len standard: {len(ordered_coordinate_list_unrefined)}, len refined: {len(ordered_coordinate_list)}")

# fourier transform selected data
data_line_list = ordered_coordinate_list
periodicity_N = len(data_line_list)
x_array = pos_to_complex(data_line_list)
k_array = dft(x_array)

len standard: 4430, len refined: 4430


In [117]:
# class to draw the fourier arm for a given frequency array
class DrawComplexFourier(Mobject):
    def __init__(self, center, k_array, scale = 1, **kwargs):
        super().__init__(**kwargs)

        self.center = center
        self.scale = scale
        self.k_array = k_array
        self.N = len(k_array)

        # previous point
        self.prior_position = np.array([0.0, 0, 0])


    # get absolute of a complex number c
    def get_absolute(self, c):
        return np.sqrt(c.real**2 + c.imag**2)
    

    # returns cirle and arrow for a given k and position
    def get_fourier_circle(self, x_k, x_n, position):
        x_k_abs = self.get_absolute(x_k) / self.N                                   # absolute of the complex value in fourier space x_k
        x_n_real = x_n.real                                                         # get the real part of the complex value in position space x_n
        x_n_imag = x_n.imag                                                         # get the imaginary of the complex value in position space x_n

        # geometrical manim objects
        circle = Circle(radius = self.scale * x_k_abs, color = GRAY, stroke_width = 0.5).move_to(position)
        arrow = Arrow(start = position, end = position + np.array([self.scale * x_n_real, self.scale * x_n_imag, 0]), color = WHITE, stroke_width = 2-x_k_abs*2/(self.N+2), buff = 0)

        # update position
        position[0] += self.scale * x_n_real
        position[1] += self.scale * x_n_imag
        return VGroup(circle, arrow)    


    # gets the n-th point of the delivered frequency array
    def get_fourier_circles(self, n, k_max):
        fourier_circles_group = VGroup()

        # zero frequency
        x_k = self.k_array[0] + self.k_array[-1]                                    # complex value in fourier space x_k
        x_n = x_k / self.N                                                          # inverse fourier transform: complex value in position space x_n

        # set the initial position (first ROTATING circle is the center)
        position = np.array([*self.center]) - np.array([self.scale * x_n.real, self.scale * x_n.imag, 0])

        # get the fourier circle and append it to the group
        fourier_circle = self.get_fourier_circle(x_k, x_n, position)
        fourier_circles_group.add(*fourier_circle)     


        # iterate through the individual non-zero frequencies
        for k in range(1, min([k_max, int(self.N/2)])):
            # +++ positive frequency
            x_k = self.k_array[k]                                                   # complex value in fourier space x_k
            x_n = x_k / self.N * np.exp(1j * 2*np.pi * k/self.N*n)                  # inverse fourier transform: complex value in position space x_n

            fourier_circle = self.get_fourier_circle(x_k, x_n, position)
            fourier_circles_group.add(*fourier_circle) 

            # --- negative freuqency
            k_neg = self.N - k                                                      # negative frequency
            x_k = self.k_array[k_neg]                                               # complex value in fourier space x_k
            x_n = x_k / self.N * np.exp(1j * 2*np.pi * k_neg/self.N*n)              # inverse fourier transform: complex value in position space x_n

            fourier_circle = self.get_fourier_circle(x_k, x_n, position)
            fourier_circles_group.add(*fourier_circle) 

        
        # draw lines or circles as you wish
        # self.add(Dot(point = position, color = BLUE, radius = 0.015))
        if n != 0:
            self.add(Line(start = self.prior_position, end = position, color = RED, stroke_width = 4))

        # update the prior for line drawings
        self.prior_position = np.array([*position])                                         
        return fourier_circles_group

In [ ]:
%%manim -qh --fps 60 $video_scene


class fe_Scene(ThreeDScene):
    def construct(self):
        CVC = Text('CVC', font_size = 12, weight = BOLD, color = WHITE, font = 'Latin Modern Sans').align_on_border(RIGHT + DOWN, buff = 0.2)
        self.add(CVC)

        center = np.array([0.0, 0.0, 0])

        # number of circles for the drawing
        fourier_order = 20


        # draw pos list
        def draw_pos_list(pos_list):
            len_pos_list = len(pos_list)
            for i in range(len_pos_list):
                self.add(Dot(point = pos_list[i]+center, color = RED, radius = 0.005))


        # draws the pos list
        # draw_pos_list(data_line_list)

        # draws the fourier image
        fourier = DrawComplexFourier(center, k_array)
        self.add(fourier)

        fourier_circles = fourier.get_fourier_circles(0, fourier_order)
        self.add(fourier_circles)

        self.wait(1.5)
        for i in range(1, periodicity_N+1):
            self.wait(1.0/60)
            self.remove(fourier_circles)
            fourier_circles = fourier.get_fourier_circles(i, fourier_order)
            self.add(fourier_circles)
        self.play(FadeOut(fourier_circles), run_time = 5)
        self.wait(5)

Output saved by overwring previous file at ../animations/fourier_elephant/fourier_elephant.mp4.


Manim Community v0.18.1

IndexError: pop from empty list

command to compress video:

ffmpeg -i cheetah_5000_1100.mp4 -c:v libx264 -pix_fmt yuv420p cheetah_5000_1100_compressed.mp4